# TASK 2

### Some basic starting stuff

In [6]:
from bs4 import BeautifulSoup
import math
from natasha import (
    Segmenter,
    MorphVocab,

    NewsEmbedding,
    NewsMorphTagger,

    Doc
)
from tqdm import tqdm
import requests
import pandas as pd

In [7]:
baseUrl = 'https://career.habr.com/vacancies?type=all'
updatableUrl = 'https://career.habr.com/vacancies?divisions[]={}&type=all&page={}'
moreBaseUrl = 'https://career.habr.com/{}'
divisions = ['backend','frontend','apps','software','testing','administration','design',
                     'management','marketing','analytics','sale','content','support','hr','telecom',
                     'other','office','security']
jobDescription = {}

In [9]:
headers = {
        "User-Agent": "Mozilla/5.0 \
               (Macintosh; Intel Mac OS X 10_10_1) \
               AppleWebKit/537.36 (KHTML, like Gecko) \
               Chrome/39.0.2171.95 Safari/537.36"
        }
links = {}

In [4]:
for div in tqdm(divisions,'parsing links...'):
    page = 1
    links[div] = []
    while True:
        r = requests.get(updatableUrl.format(div,str(page)),headers=headers)

        if not r.ok:
            break

        soup = BeautifulSoup(r.content, 'html.parser')
        vacancyCard = soup.find_all('div','vacancy-card')

        if vacancyCard == []:
            break

        for card in vacancyCard:
            vacancyLink = card.find('div','vacancy-card__title').a['href']
            links[div].append(vacancyLink)


        page += 1

parsing links...: 100%|██████████| 18/18 [03:50<00:00, 12.79s/it]


In [20]:
linksSaved = links.copy()
maxLen = 0
for div in divisions:
    # print(len(links[div]), div)
    if len(links[div]) > maxLen:
        maxLen = len(links[div])
# print(maxLen)
for div in divisions:
    while len(links[div]) != maxLen:
        links[div].append('NaN')
linkDict = pandas.DataFrame(data=links)
linkDict.to_pickle('linksDict')

In [4]:
tmpDataFrame = pd.read_pickle('linksDict')
tmpDataFrame

FileNotFoundError: [Errno 2] No such file or directory: 'linksDict'

In [ ]:
link = moreBaseUrl.format(links['backend'][0])
r = requests.get(link, headers=headers)
soup = BeautifulSoup(r.content,'html.parser')
description = soup.find('div',"job_show_description__vacancy_description").find('div','style-ugc').text
print(description)

## Parsing job descriptions

In [8]:
for division in tqdm(links):
    jobDescription[division] = []
    for link in links[division]:
        r = requests.get(moreBaseUrl.format(link),headers=headers)
        if not r.ok:
            break
        soup = BeautifulSoup(r.content,'html.parser')
        if soup.find('div',"job_show_description__vacancy_description") != []:
            description = soup.find('div',"job_show_description__vacancy_description").find('div','style-ugc').text
        else:
            break
        jobDescription[division].append(description)

100%|██████████| 18/18 [49:38<00:00, 165.48s/it] 


## Saving up data

In [9]:
maxLen = 0
for div in divisions:
    if len(jobDescription[div]) > maxLen:
        maxLen = len(jobDescription[div])
maxLen
for div in divisions:
    while len(jobDescription[div]) != maxLen:
        jobDescription[div].append('nan')

In [24]:
data = pd.read_csv('jobDescription.csv', encoding='utf-32', index_col='Unnamed: 0')

In [25]:
data.to_pickle('jobDescription')

In [10]:
savedJobDescription = pd.read_pickle('jobDescription')

In [11]:
savedJobDescription

,backend,frontend,apps,software,testing,administration,design,management,marketing,analytics,sale,content,support,hr,telecom,other,office,security
0,Команда Easy Development занимается разработко...,Мы динамично растущая международная SaaS компа...,Банк Открытие ищет Android разработчика (Senio...,Наша команда в поисках JavaScript/Node.js разр...,EVRAZ — один из крупнейших производителей стал...,Мы — растущая международная компания из Хельси...,Мы делаем Apphud – инфраструктуру для монетиза...,Osome is a Fintech startup with offices in Lon...,На данной позиции предстоит:Настройка систем м...,We have a highly-loaded service with 740M requ...,"HiPoHeads - IT Рекрутинговое агентство, на уро...","Мы приглашаем инженера по локализации, желающе...",Вам предстоит:\n Осуществлять поддержк...,VOLKA HR CREW / Recruiter (Russian Speaking...,"Обязанности:\nНастройка, тестирование, испытан...",Мы отвечаем за UX (пользовательский опыт) в бо...,Должностные обязанности:\n1. Административная ...,"Ищем\n человека, который имеет опыт с разработ..."
1,"В нашу команду, работающую над высоконагруженн...",ПРОЕКТ:Мы занимаемся Программой лояльности НСП...,Банк Открытие ищет в команду IOS разработчика ...,We are looking for a Salesforce Developer to c...,Привет!Мы активно растущая SaaS компания. Рабо...,\n\n«РТЛабс» - ОДНА ИЗ ВЕДУЩИХ IT-КОМПАНИЙ РОС...,Основные требования:\nSoft skills\n Хор...,В настоящее время мы ищем Project Manager для ...,Что Вы будете у нас делать:Определять виденье ...,Банк Открытие ищет Cистемного аналитика (урове...,Взаимодействовать с партнёрами ЛК на выделенно...,ВНИМАНИЕ: отклик лучше прислать через форму на...,"Вам предстоит:\n Консультация, поддерж...","Компания «Ай-Теко» — резидент «Сколково», веду...",Кто мы и чем занимаемся:Компания Eltex - это о...,На данный момент мы находимся в поиске стажера...,NaN,Rambler&Co занимает первое место среди медиахо...
2,Мы — растущая международная компания из Хельси...,Наша компания занимается написанием ПО для суд...,"MGVC — Инвестиционное подразделение MY.GAMES, ...",Привет!\nМы разыскиваем сильного Senior FullSt...,ПРОЕКТ:Мы разрабатываем Клиринговую систему дл...,Компания Билайн ищет Администратора БД Postgre...,Лента Онлайн - это молодой и быстро развивающи...,"В настоящее время мы ищем Project Manager, кот...",Мы строим «Bloomberg terminal for DeFi space» ...,\n\n«РТЛабс» - ОДНА ИЗ ВЕДУЩИХ IT-КОМПАНИЙ РОС...,Сфера деятельности Департамента систем управле...,"Привет, мы — команда Icons8. Мы рисуем иконки ...",Основные задачи:\nучастие в проектах по создан...,"At Cube Dev, we are building a technology stac...",NaN,В группу развития технологий поиска уязвимосте...,NaN,Международная спортивная e-commerce компания и...
3,Друзья! Разыскиваем сильного и самостоятельног...,Ищем сотрудника на удаленную работу PHP програ...,Ищем junior - Middle Android developer\n\n\n\n...,"Привет!\nМеня зовут Мика, и я руковожу командо...","FUNCORP — международная компания, c 2004 года ...",EVRAZ — один из крупнейших производителей стал...,Ищем в команду дизайнера делать полезные и удо...,Delivery Club — лидер рынка foodtech в России....,Агентство интернет рекламы Impecco ищет специа...,Osome is a Fintech startup with offices in Lon...,"Человек, которого ищем, умеет и любит налажива...",Издательство «Питер» — лидер на рынке професси...,"Компания UserGate, производитель систем в обла...",Postoplan.app — международная платформа маркет...,NaN,Команда поддержки и администрирования финансов...,NaN,"Задачи, которые необходимо закрывать:\nФормиро..."
4,Наша команда занимается разработкой и поддержк...,Компания Софт Дивижн (www.soft-division.ru) ищ...,Мы сейчас расширяем одну из команд направления...,\n\n\n\n«РТЛабс» - ОДНА ИЗ ВЕДУЩИХ IT-КОМПАНИЙ...,Обязанности:\nавтоматизация тестирования бэк-о...,КОМАНДА:Команда инженеров по автоматизации про...,Текущая задача:Закончить разработку мобильного...,Привет! Ищем РМа на веб-проекты!\nЧто нужно де...,"Senior Product Marketing Manager, Go-to-market...",ЕВРАЗ – междуна

# TF-IDF

In [10]:
# import math

In [19]:
# dictionary = {}
# numOfDocs = 0
# for div in divisions:
#     numOfDocs += len(savedJobDescription[div])
# for div in divisions:
#     for description in savedJobDescription[div]:
#         for word in description:
#             if word not in dictionary:


TypeError: list indices must be integers or slices, not str

## Preprocessing with Natasha

#### experimenting for the first job description

In [162]:
text = savedJobDescription['frontend'][431]
a = pd.isnull(savedJobDescription.at[431,'frontend'])
if a == False:
    print('whyyy?')
    doc = Doc(text)
    emb = NewsEmbedding()

    morph_vocab = MorphVocab()
    morph_tagger = NewsMorphTagger(emb)
    segmnetr = Segmenter()

    doc.segment(segmnetr)
    doc.tag_morph(morph_tagger)

    for token in doc.tokens:
        token.lemmatize(morph_vocab)

    # {_.text: _.lemma for _ in doc.tokens}

    doc.tag_morph(morph_tagger)
    lemmatizedNUsedulWords = []
    for token in doc.tokens:
        if token.pos != 'PUNCT' and token.pos != 'PRON' and token.pos != 'CCONJ' and token.pos != 'DET' and \
            token.pos != 'ADP':
            lemmatizedNUsedulWords.append(token.lemma)



    # doc.sents[0].morph.print()
    print(lemmatizedNUsedulWords)
print('nice')

nice


In [29]:
for division in tqdm(divisions,'preprocessing data...'):
    counter = 0
    for description in savedJobDescription[division]:
        if pd.isnull(savedJobDescription.at[counter,division]) == False:
            doc = Doc(description)
            emb = NewsEmbedding()

            morph_vocab = MorphVocab()
            morph_tagger = NewsMorphTagger(emb)
            segmentr = Segmenter()

            doc.segment(segmentr)
            doc.tag_morph(morph_tagger)

            for token in doc.tokens:
                token.lemmatize(morph_vocab)

            doc.tag_morph(morph_tagger)
            preProcessedBoW = []

            for token in doc.tokens:
                if token.pos != 'PUNCT' and token.pos != 'PRON' and token.pos != 'CCONJ' and token.pos != 'DET' and token.pos != 'ADP':
                    preProcessedBoW.append(token.lemma)
            savedJobDescription[division][counter] = preProcessedBoW
            counter += 1
        else:
            savedJobDescription[division][counter] = ['']
            counter += 1

preprocessing data...: 100%|██████████| 18/18 [32:43<00:00, 109.07s/it] 


In [31]:
savedJobDescription.to_pickle('processedToeknizedLemmatizedDescriptions')
savedJobDescription['backend'][0]

['команда',
 'easy',
 'development',
 'заниматься',
 'разработка',
 'высоко-нагрузить',
 'онлайн',
 'приложение',
 'real-time',
 'bidding',
 'dsp',
 'ssp',
 'soap',
 'smmp',
 'биллинга',
 'мобильный',
 'приложение',
 'социальный',
 'сеть',
 'много',
 'другой',
 'работать',
 'jira',
 'confluence',
 'связь',
 'использовать',
 'zoom',
 'telegram',
 'быть',
 'devops',
 'технический',
 'менеджер',
 'scrum-мастер',
 'искать',
 'middle',
 'middle',
 '+',
 'python',
 'разработчик',
 'разработка',
 'бэкенда',
 'международный',
 'социальный',
 'сеть',
 'абсолютно',
 'новый',
 'проект',
 'быть',
 'исток',
 'проект',
 'участвовать',
 'построение',
 'архитектура',
 'backend',
 'python',
 'быть',
 'обслуживать',
 'клиент',
 'web',
 'ios',
 'android',
 'приложение',
 'требование',
 'опыт',
 '2',
 'год',
 'продуктовый',
 'разработкиопыт',
 'работа',
 'команда',
 'разработка',
 '1',
 'годавысокий',
 'уровень',
 'знание',
 'python',
 'django',
 'понимание',
 'методология',
 'tdd',
 'понимание',
 'rest',

In [35]:
dataProccesed = pd.read_pickle('C:\Users\Gena\AppData\Roaming\JetBrains\JetBrainsDS2021.3\projects\HSE\UsefulData')

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-35-02067e43d327>, line 1)

In [36]:
dataProccesed['software'][18]

NameError: name 'dataProccesed' is not defined

In [4]:
jobDescription = pd.read_pickle('jobDescription')
jobDescription['backend'][0].split()

['Команда',
 'Easy',
 'Development',
 'занимается',
 'разработкой',
 'высоко-нагруженных',
 'онлайн',
 'приложений(Real-Time',
 'Bidding,',
 'DSP/SSP,',
 'SOAP/SMMP',
 'биллинги),',
 'мобильных',
 'приложений,',
 'социальных',
 'сетей',
 'и',
 'много',
 'чего',
 'другого.',
 'Мы',
 'работаем',
 'в',
 'Jira/Confluence,',
 'для',
 'связи',
 'используем',
 'Zoom/Telegram.',
 'У',
 'нас',
 'есть',
 'DevOps,',
 'технические',
 'менеджеры',
 'и',
 'Scrum-мастера.',
 'Ищем',
 'Middle/Middle+',
 'Python',
 'разработчика',
 'для',
 'разработки',
 'бэкенда',
 'международной',
 'социальной',
 'сети.',
 'Это',
 'абсолютно',
 'новый',
 'проект.',
 'Вы',
 'будете',
 'у',
 'истоков',
 'проекта',
 'и',
 'участвовать',
 'в',
 'построении',
 'архитектуры.',
 'Backend',
 'на',
 'Python',
 'будет',
 'обслуживать',
 'клиентов',
 'в',
 'WEB,',
 'iOS,',
 'Android',
 'приложениях.',
 'Требования',
 'Опыт',
 'от',
 '2',
 'лет',
 'продуктовой',
 'разработкиОпыт',
 'работы',
 'в',
 'команде',
 'разработки',
 'от

In [8]:
freqDict = {}
for div in tqdm(divisions, 'counting words...'):
    for iter in range(0, len(jobDescription[div])):
        if pd.isnull(jobDescription.at[iter,div]) == False:
            for word in jobDescription[div][iter].split():
                if word not in freqDict:
                    freqDict[word] = 1
                else:
                    freqDict[word] += 1
freqDict

counting words...: 100%|██████████| 18/18 [00:00<00:00, 58.43it/s]


{'Команда': 265,
 'Easy': 4,
 'Development': 61,
 'занимается': 274,
 'разработкой': 351,
 'высоко-нагруженных': 4,
 'онлайн': 140,
 'приложений(Real-Time': 4,
 'Bidding,': 4,
 'DSP/SSP,': 4,
 'SOAP/SMMP': 4,
 'биллинги),': 4,
 'мобильных': 406,
 'приложений,': 127,
 'социальных': 57,
 'сетей': 74,
 'и': 40457,
 'много': 275,
 'чего': 72,
 'другого.': 3,
 'Мы': 3760,
 'работаем': 448,
 'в': 24506,
 'Jira/Confluence,': 5,
 'для': 7483,
 'связи': 273,
 'используем': 349,
 'Zoom/Telegram.': 2,
 'У': 725,
 'нас': 1807,
 'есть': 943,
 'DevOps,': 46,
 'технические': 179,
 'менеджеры': 15,
 'Scrum-мастера.': 2,
 'Ищем': 215,
 'Middle/Middle+': 1,
 'Python': 365,
 'разработчика': 312,
 'разработки': 3026,
 'бэкенда': 31,
 'международной': 176,
 'социальной': 23,
 'сети.': 18,
 'Это': 189,
 'абсолютно': 21,
 'новый': 218,
 'проект.': 23,
 'Вы': 413,
 'будете': 135,
 'у': 1226,
 'истоков': 15,
 'проекта': 331,
 'участвовать': 258,
 'построении': 48,
 'архитектуры.': 33,
 'Backend': 97,
 'на': 11

In [9]:
freqDict

{'Команда': 265,
 'Easy': 4,
 'Development': 61,
 'занимается': 274,
 'разработкой': 351,
 'высоко-нагруженных': 4,
 'онлайн': 140,
 'приложений(Real-Time': 4,
 'Bidding,': 4,
 'DSP/SSP,': 4,
 'SOAP/SMMP': 4,
 'биллинги),': 4,
 'мобильных': 406,
 'приложений,': 127,
 'социальных': 57,
 'сетей': 74,
 'и': 40457,
 'много': 275,
 'чего': 72,
 'другого.': 3,
 'Мы': 3760,
 'работаем': 448,
 'в': 24506,
 'Jira/Confluence,': 5,
 'для': 7483,
 'связи': 273,
 'используем': 349,
 'Zoom/Telegram.': 2,
 'У': 725,
 'нас': 1807,
 'есть': 943,
 'DevOps,': 46,
 'технические': 179,
 'менеджеры': 15,
 'Scrum-мастера.': 2,
 'Ищем': 215,
 'Middle/Middle+': 1,
 'Python': 365,
 'разработчика': 312,
 'разработки': 3026,
 'бэкенда': 31,
 'международной': 176,
 'социальной': 23,
 'сети.': 18,
 'Это': 189,
 'абсолютно': 21,
 'новый': 218,
 'проект.': 23,
 'Вы': 413,
 'будете': 135,
 'у': 1226,
 'истоков': 15,
 'проекта': 331,
 'участвовать': 258,
 'построении': 48,
 'архитектуры.': 33,
 'Backend': 97,
 'на': 11

In [42]:
import heapq
mostFreqDict = heapq.nlargest(int(len(freqDict)-len(freqDict)*0.3), freqDict, key=freqDict.get)
mostFreqDict

['',
 'работа',
 'опыт',
 'разработка',
 'команда',
 'знание',
 'быть',
 'проект',
 'компания',
 'система',
 'задача',
 'работать',
 'and',
 'продукт',
 'возможность',
 'офис',
 'новый',
 'год',
 'решение',
 '+',
 'не',
 'данные',
 'приложение',
 'тестирование',
 'развитие',
 'требование',
 'понимание',
 'the',
 'сервис',
 'технический',
 'процесс',
 'to',
 'умение',
 'of',
 'разработчик',
 'уровень',
 'технология',
 'как',
 'корпоративный',
 'участие',
 'обучение',
 'сотрудник',
 '2',
 'код',
 'язык',
 'профессиональный',
 'день',
 'рабочий',
 'in',
 'другой',
 'принцип',
 'a',
 'искать',
 'график',
 'предлагать',
 'хороший',
 'навык',
 'архитектура',
 'если',
 'поддержка',
 'клиент',
 'плюс',
 'так',
 'условие',
 'платформа',
 'английский',
 'что',
 'проектирование',
 'sql',
 'гибкий',
 'заниматься',
 'дмс',
 'человек',
 'разрабатывать',
 'with',
 'современный',
 'управление',
 'мобильный',
 'пользователь',
 'java',
 'анализ',
 'заработный',
 '1',
 'использование',
 'интересный',
 'р

In [10]:
len(freqDict)

117578

In [11]:
len(mostFreqDict)

NameError: name 'mostFreqDict' is not defined

In [46]:
wordDict = pandas.DataFrame(data=mostFreqDict)
wordDict.to_pickle('wordDict')

In [11]:
import pandas as pd
data = pd.read_pickle('namesDict')
listtt = data[0].to_list()
listtt
jobDescription = pd.read_pickle('jobDescription')

In [13]:
import tqdm

In [17]:
vecrotizedCorpus = {}
for div in tqdm(divisions, 'vectorizing'):
    vecrotizedCorpus[div] = []
    for description in jobDescription[div]:
        tmpVector = []
        for token in listtt:
            if token in description:
                tmpVector.append(1)
            else:
                tmpVector.append(0)
        vecrotizedCorpus[div].append(tmpVector)

TypeError: 'module' object is not callable

In [49]:
vecrotizedCorpus['backend'][0].count(1)

111

In [5]:
dataVectorized = pandas.DataFrame(data=vecrotizedCorpus)
dataVectorized.to_pickle('vectorizedSet')

NameError: name 'pandas' is not defined

In [ ]:
# vectorizedSet - for comparing vector
# wordDict - for creating vectors from inputLink
# linksDict - for giving out link on vacancy
# JobDescription - for giving description if there is no Link due to "link expired"
# all data is from 29.10.2021 16:26 from https://www.career.habr.ru

In [51]:
# This ipython notebook was used only to gain all data needed for implementing task 2 as a script

In [55]:
listOfWords = wordDict[0].to_list()

In [56]:
listOfWords

['',
 'работа',
 'опыт',
 'разработка',
 'команда',
 'знание',
 'быть',
 'проект',
 'компания',
 'система',
 'задача',
 'работать',
 'and',
 'продукт',
 'возможность',
 'офис',
 'новый',
 'год',
 'решение',
 '+',
 'не',
 'данные',
 'приложение',
 'тестирование',
 'развитие',
 'требование',
 'понимание',
 'the',
 'сервис',
 'технический',
 'процесс',
 'to',
 'умение',
 'of',
 'разработчик',
 'уровень',
 'технология',
 'как',
 'корпоративный',
 'участие',
 'обучение',
 'сотрудник',
 '2',
 'код',
 'язык',
 'профессиональный',
 'день',
 'рабочий',
 'in',
 'другой',
 'принцип',
 'a',
 'искать',
 'график',
 'предлагать',
 'хороший',
 'навык',
 'архитектура',
 'если',
 'поддержка',
 'клиент',
 'плюс',
 'так',
 'условие',
 'платформа',
 'английский',
 'что',
 'проектирование',
 'sql',
 'гибкий',
 'заниматься',
 'дмс',
 'человек',
 'разрабатывать',
 'with',
 'современный',
 'управление',
 'мобильный',
 'пользователь',
 'java',
 'анализ',
 'заработный',
 '1',
 'использование',
 'интересный',
 'р

In [12]:
import io
import re
import string
import tqdm

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

In [13]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [14]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE
vocab = freqDict

In [15]:
inverse_vocab = {index: token for token, index in vocab.items()}
print(inverse_vocab)

{265: 'сложные', 4: 'Authorizations', 61: 'интеграционного', 274: 'занимается', 351: 'разработкой', 140: 'Our', 406: 'мобильных', 127: 'модели', 57: 'marketing', 74: 'SDK', 40457: 'и', 275: 'владение', 72: 'расширяем', 3: 'GRC,', 3760: 'Мы', 448: 'работаем', 24506: 'в', 5: 'осведомленности', 7483: 'для', 273: 'современных', 349: 'используем', 2: 'programs,', 725: 'наших', 1807: 'нас', 943: 'есть', 46: 'контента', 179: 'больше', 15: 'Заказчиком', 215: '&', 1: 'granted', 365: 'Python', 312: 'REST', 3026: 'разработки', 31: 'Tableau', 176: 'skills', 23: 'helps', 18: 'Ручное', 189: 'помощь', 21: 'презентовать', 218: 'этом', 413: 'Вы', 135: 'современный', 1226: 'у', 331: 'России', 258: 'испытательного', 48: 'Apple', 33: 'отчеты', 97: '1С', 11722: 'на', 1189: 'будет', 358: 'уровня', 9: 'Бизнес-Требований', 41: 'automation', 303: 'интеграции', 8: 'vmware', 187: 'России.', 2612: 'Опыт', 4287: 'от', 627: '2', 561: 'лет', 198: 'помощью', 11: 'аналитика)', 7952: 'работы', 974: 'команде', 443: '1',